<a href="https://colab.research.google.com/github/gamountainmama/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns='EIN', axis=1)

application_df = application_df.drop(columns='NAME', axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_count = application_df['APPLICATION_TYPE'].value_counts()
app_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_count[app_count<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df['CLASSIFICATION'].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count_large = application_df['CLASSIFICATION'].value_counts().loc[lambda x: x>1]
class_count_large

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
application_df = application_df.drop(columns='STATUS', axis=1)
#application_df = application_df.drop(columns='SPECIAL_CONSIDERATIONS', axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [10]:
application_df['ASK_AMT'].describe()

count    3.429900e+04
mean     2.769199e+06
std      8.713045e+07
min      5.000000e+03
25%      5.000000e+03
50%      5.000000e+03
75%      7.742000e+03
max      8.597806e+09
Name: ASK_AMT, dtype: float64

In [11]:
bins = [0,5000,10000,20000,30000,40000,50000,100000,1000000,10000000000]
labels = ['$0-$5,000','$5,001-$10,000','$10,001-$20,000','$20,001-$30,000','$30,001-$40,000','$40,001-$50,000','$50,001-$100,000','$100,001-$1,000,000','$1,000,000+']
application_df['ASK_AMT'] = pd.cut(application_df['ASK_AMT'], bins=bins, labels=labels)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,"$0-$5,000",1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,"$100,001-$1,000,000",1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,"$0-$5,000",0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,"$5,001-$10,000",1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,"$100,001-$1,000,000",1


In [12]:
amt_count = application_df['ASK_AMT'].value_counts()
amt_count

$0-$5,000              25398
$100,001-$1,000,000     2954
$1,000,000+             1577
$50,001-$100,000        1423
$10,001-$20,000          789
$20,001-$30,000          655
$5,001-$10,000           549
$30,001-$40,000          523
$40,001-$50,000          431
Name: ASK_AMT, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
num_data = pd.get_dummies(application_df)
num_data.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,SPECIAL_CONSIDERATIONS_Y,"ASK_AMT_$0-$5,000","ASK_AMT_$5,001-$10,000","ASK_AMT_$10,001-$20,000","ASK_AMT_$20,001-$30,000","ASK_AMT_$30,001-$40,000","ASK_AMT_$40,001-$50,000","ASK_AMT_$50,001-$100,000","ASK_AMT_$100,001-$1,000,000","ASK_AMT_$1,000,000+"
0,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
# Split our preprocessed data into our features and target arrays
X = num_data.drop('IS_SUCCESSFUL', axis=1).values
y = num_data['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input = X_train_scaled.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=120, activation="relu", input_dim=input))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=125, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=130, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               6120      
                                                                 
 dense_1 (Dense)             (None, 125)               15125     
                                                                 
 dense_2 (Dense)             (None, 130)               16380     
                                                                 
 dense_3 (Dense)             (None, 1)                 131       
                                                                 
Total params: 37,756
Trainable params: 37,756
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [18]:
# Train the model
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.hdf5", monitor='accuracy', verbose=1,
    save_best_only=True, mode='auto', save_freq=5)

fit_model = nn.fit(X_train_scaled, y_train, epochs=50, verbose=1, callbacks=[checkpoint])

Streaming output truncated to the last 5000 lines.
Epoch 35: accuracy did not improve from 0.84375
446/804 [===============>..............] - ETA: 1s - loss: 0.1755 - accuracy: 0.7465
Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375
460/804 [================>.............] - ETA: 1s - loss: 0.1754 - accuracy: 0.7469
Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375
474/804 [================>.............] - ETA: 1s - loss: 0.1753 - accuracy: 0.7470
Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375
487/804 [=================>............] - ETA: 1s - loss: 0.1753 - accuracy: 0.7467
Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375
499/804 [=================

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 [==============================] - 1s 2ms/step - loss: 0.1883 - accuracy: 0.7257
Loss: 0.18830981850624084, Accuracy: 0.7257142663002014


In [20]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')

from google.colab import files
files.download('AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>